In [1]:
!pip install numpy==1.19.5 --user

In [2]:
!pip install h5py==2.10.0 --user

In [3]:
!pip install gensim==3.6.0 --user

In [4]:
!pip install deepcut --user

In [5]:
!pip install pythainlp --user

In [6]:
!pip install nltk --user

In [7]:
from pythainlp.corpus import thai_stopwords
import deepcut
from time import time
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
import datetime
import tensorflow
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Lambda
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.callbacks import ModelCheckpoint
import difflib

C:\Users\NATTHAWATTUNGRUETHAI\AppData\Roaming\Python\Python38\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [8]:
from flask_ngrok import run_with_ngrok
from flask import Flask, jsonify, request
import json

In [9]:
#Clean Text
def remove_repettition(text):
    token_list = list(text)
    if len(token_list) > 2:
        filter_list = [True, True]
        n = len(token_list)
        for i in range(2, n):
            if (token_list[i] == token_list[i-1]) and (token_list[i] == token_list[i-2]):
                filter_list.append(False)
            else:
                filter_list.append(True)

        output = ''.join(np.array(token_list)[filter_list])
    else:
        output = text
    return output

def cleansing(text):
    # \t, \n, \xa0 and other special characters. Replace by blank string
    text = re.sub('[\t\n\xa0\"\'!?\/\(\)%\:\=\-\+\*\_ๆ]', '', text)
    
    # Numbers. Replace by space
    text = re.sub('[0-9]', ' ', text)
    
    # Dot. Replace by space
    text = re.sub('[\.]', ' ', text)
    
    # One or more consecutive space. Replace by single space
    text = re.sub('\s+',' ',text)
    
    # Remove 2 or more repettition
    text = remove_repettition(text)
    
    return text

In [10]:
import gensim
wv_model = gensim.models.Word2Vec.load('corpus.th.model')

In [11]:
def word2idx(word):
    index = 0
    index = wv_model.wv.vocab[word].index
    return index

In [12]:
def word_index(listword):
    dataset = []
    vocabulary = dict()
    inverse_vocabulary = ['<unk>']  # '<unk>' will never be used, it is only a placeholder for the [0, 0, ....0] embedding
    for sentence in listword:
        tmp = []
        for w in sentence:
            if w not in wv_model:
                continue

            if w not in vocabulary:
                vocabulary[w] = len(inverse_vocabulary)
                tmp.append(len(inverse_vocabulary))
                inverse_vocabulary.append(w)
            else:
                tmp.append(word2idx(w))
        dataset.append(tmp)
    return np.array(dataset)

In [13]:
# define word embedding
vocab_list = [(k, wv_model.wv[k]) for k, v in wv_model.wv.vocab.items()]
embeddings_matrix = np.zeros((len(wv_model.wv.vocab.items()) + 1, wv_model.vector_size))
for i in range(len(vocab_list)):
    word = vocab_list[i][0]
    embeddings_matrix[i + 1] = vocab_list[i][1]

In [14]:
# vocab_list

In [15]:
EMBEDDING_DIM = 300
embeddings_matrix = 1 * np.random.randn(len(vocab_list) + 1, EMBEDDING_DIM)  # This will be the embedding matrix
embeddings_matrix[0] = 0  # So that the padding will be ignored

In [16]:
# Model variables
n_hidden = 256
batch_size = 128
n_epoch = 100
max_seq_length = 2704

In [17]:
# embeddings_matrix

In [18]:
def exponent_neg_manhattan_distance(left, right):
    ''' Helper function for the similarity estimate of the LSTMs outputs'''
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))

In [19]:
# The visible layer
left_input = Input(shape=(max_seq_length,), dtype='int32')
right_input = Input(shape=(max_seq_length,), dtype='int32')

embedding_layer = Embedding(len(embeddings_matrix), EMBEDDING_DIM, weights=[embeddings_matrix], input_length=max_seq_length, trainable=False)

# Embedded version of the inputs
encoded_left = embedding_layer(left_input)
encoded_right = embedding_layer(right_input)

# Since this is a siamese network, both sides share the same LSTM
shared_lstm = LSTM(n_hidden)

left_output = shared_lstm(encoded_left)
right_output = shared_lstm(encoded_right)

# Calculates the distance as defined by the MaLSTM model
malstm_distance = Lambda(function=lambda x: exponent_neg_manhattan_distance(x[0], x[1]),output_shape=lambda x: (x[0][0], 1))([left_output, right_output])

# Pack it all up into a model
malstm = Model([left_input, right_input], [malstm_distance])


malstm.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

# Start training
training_start_time = time()

In [20]:
malstm.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 2704)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 2704)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 2704, 300)    9468300     input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     (None, 256)          570368      embedding[0][0]              

In [21]:
# Load best weight from model
malstm.load_weights('sm_colab_ka.h5')

#Test with Text

In [22]:
def prepare_for_predict(input_questions):
    q_input= []
    cleansing(input_questions)
    tokenized_input_1 =deepcut.tokenize(input_questions)
    for sentence in tokenized_input_1:
      q_input.append(sentence)
    q_input= word_index(tokenized_input_1)
    q_input = pad_sequences(q_input, maxlen=max_seq_length)
    return q_input

In [23]:
max_word = 19219
max_seq_length = 2704

In [24]:
from itertools import chain
def tokenize_text_list(ls):
    """Tokenize list of text"""
    return list(chain.from_iterable([deepcut.tokenize(ls)]))

In [25]:
#Duplicate list
def duplicate(testList, n):
    return [ele for ele in testList for _ in range(n)]

In [53]:
app = Flask(__name__)
@app.route('/prediction', methods=['POST', 'GET'])
def home():
    if request.method == 'POST':
        data = request.get_json()
        questions = data["questions"]
        inputQuestion = data["inputQuestion"]
        print('input: ' + inputQuestion)
        newQuestion = {'question1':questions}
        questionsDB = pd.DataFrame(data=newQuestion)
        tokenized_category =questionsDB.question1.map(tokenize_text_list)
#         print(tokenized_category)
        max_word = 19219
        max_seq_length = 2704
        q_category= []
        for sentence in tokenized_category:
            q_category.append(sentence)
        q_category = word_index(q_category)
        all_Question_categorylen = len(q_category)
        all_Question_categorylen
        
        tokenized_input_2 = cleansing(inputQuestion)
        tokenized_input_2 =deepcut.tokenize(tokenized_input_2)
        
        tokenized_dup_input_2= duplicate([tokenized_input_2],all_Question_categorylen)
        q_user = word_index(tokenized_dup_input_2)
        # Split to dicts
        M_input = {'left': q_category, 'right': q_user}
        # Zero padding
        for model_input, side in itertools.product([M_input], ['left', 'right']):
            model_input[side] = pad_sequences(model_input[side], maxlen=max_seq_length)

        # Make sure everything is ok
        assert M_input['left'].shape == M_input['right'].shape
        play_predict = malstm.predict(x=[M_input['left'],  M_input['right']])
        max_question_percentage = max(play_predict)
        question_index = np.where(play_predict == max_question_percentage)
        predictedQuestion = questionsDB.loc[question_index[0][0],'question1']
        print('output: '+ predictedQuestion+ ' ' + "%lf" % max_question_percentage)
        value = {
          "predictedQuestion": predictedQuestion,
          "similarity": "%lf" % max_question_percentage
        }
        return json.dumps(value, ensure_ascii=False).encode('utf8')
    else:
        return "Hello. I am alive!"

app.run(port=3000,debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:3000/ (Press CTRL+C to quit)


input: วิศวคอมมีหลักสูตรอะไรบ้าง


<ipython-input-12-e36182fd4f37>:8: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if w not in wv_model:
<ipython-input-12-e36182fd4f37>:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(dataset)
127.0.0.1 - - [13/May/2021 18:03:06] "POST /prediction HTTP/1.1" 200 -


output: วิศวะคอม/ มีทั้งหมดกี่หลักสูตร? มีหลักสูตรอะไรบ้าง? 0.996301


# example json of api

In [ ]:
{"questions":["วิชา CPE332/Professional issuesมีวิชาตัวต่อมั้ยครับ?", "วิชาเลือกที่มีวิชาบังคับลงก่อน หลักสูตรปกติ?", "วิศวะคอม/หลักสูตรนานาชาติ ปี 4 เทอม 2\nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?", "วิศวะคอม/หลักสูตรวิทยาศาสตรบัณฑิต สาขาวิชาวิทยาศาสตร์ข้อมูลสุขภาพ \nปี 4 เทอม 2 ต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?", "วิชาเลือกที่มีวิชาบังคับลงก่อน หลักสูตรวิทยาศาสตรบัณฑิต สาขาวิชาวิทยาศาสตร์ข้อมูลสุขภาพ ?", "วิชา CPE375/Interactive computingมีวิชาตัวต่อมั้ยครับ?", "หน่วยกิตที่ต้องเก็บให้ครบก่อนจบหรือไม่?", "วิศวะคอม/วิศวกรรมคอมพิวเตอหลักสูตรวิทยาศาสตรบัณฑิต สาขาวิชาวิทยาศาสตร์ข้อมูลสุขภาพ \nปี 3 เทอม 2 ต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?", "วิชา CPE224/Computer architectures มีวิชาตัวต่อมั้ยครับ?", "วิศวะคอม/หลักสูตรนานาชาติปี 1 เทอม 2\nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?", "วิศวะคอม/หลักสูตรนานาชาติปี 1 เทอม 1 \nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?", "วิชาเลือกที่มีวิชาบังคับลงก่อน หลักสูตรนานาชาติ?", "วิศวกรรมคอมพิวเตอร์หลักสูตรวิทยาศาตร์ข้อมูลสุขภาพ มีกี่หน่วยกิต? ", "วิศวกรรมคอมพิวเตอร์หลักสูตรปกติ ปีที่1-2 เรียนอะไรบ้าง?", "วิศวะคอม/หลักสูตรนานาชาติปี 2 เทอม 2\nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?", "วิชา CPE212/Algorithm มีวิชาตัวต่อมั้ยครับ?", "วิศวะคอม/หลักสูตรปกติ  ปี 4 เทอม 1\nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?", "วิศวะคอมหลักสูตรวิทยาศาตร์ข้อมูลสุขภาพมีอัตราค่าเรียนเท่าไหร่", "วิศวะคอม/หลักสูตรวิทยาศาสตรบัณฑิต สาขาวิชาวิทยาศาสตร์ข้อมูลสุขภาพ \nปี 3 เทอม 1 ต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?", "วิชาภาคบังคับที่มีวิชาต่อเนื่อง มีอะไรบ้าง หลักสูตรปกติ?", "วิชา CPE314/Computer networksมีวิชาตัวต่อมั้ยครับ?", "วิศวกรรมคอมพิวเตอร์หลักสูตรเรสิเดนทอล คอเลจ ปีที่ 1-2 เรียนอะไรบ้าง?", "วิศวกรรมคอมพิวเตอร์หลักสูตรวิทยาศาตร์ข้อมูลสุขภาพ ปีที่ 3 เรียนอะไรบ้าง?", "วิชา CPE327/Software engineering มีวิชาตัวต่อมั้ยครับ?", "วิศวะคอมพิวเตอร์แต่ละหลักสูตรมีระยะเวลาในการศึกษากี่ปี?", "วิชา CPE325/Big data มีวิชาตัวต่อมั้ยครับ?", "วิชา CPE111/Data Structure มีวิชาตัวต่อมั้ยครับ?", "วิศวะคอม/หลักสูตรปกติ ปี 2 เทอม 1\nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?", "วิศวะคอม/หลักสูตรนานาชาติ ปี 2 เทอม 1\nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?", "วิชาเลือกเสรีต้องลงวิชานอกภาคเท่านั้นหรือไม่?", "วิศวะคอมหลักสูตรปกติมีอัตราค่าเรียนเท่าไหร่", "วิชา CPE326/Operating systems มีวิชาตัวต่อมั้ยครับ?", "วิชา CPE121/Discrete มีวิชาตัวต่อมั้ยครับ?", "วิศวกรรมคอมพิวเตอร์หลักสูตรวิทยาศาตร์ข้อมูลสุขภาพ ปีที่ 4 เรียนอะไรบ้าง?", "วิศวกรรมคอมพิวเตอร์หลักสูตรปกติ มีกี่หน่วยกิต?", "วิชา CPE342/Java programmingมีวิชาตัวต่อมั้ยครับ?", "วิชาภาษาอังกฤษ LNG มีเกณท์การเรื่มเรียนอย่างไร หลักสูตรวิทยาศาสตรบัณฑิต\nสาขาวิชาวิทยาศาสตร์ข้อมูลสุขภาพ ?", "วิชา CPE343/Object orientedมีวิชาตัวต่อมั้ยครับ?", "วิชา CPE100/Programming มีวิชาตัวต่อมั้ยครับ?", "วิศวกรรมคอมพิวเตอร์หลักสูตรนานาชาติ มีกี่หน่วยกิต?", "วิศวะคอม/หลักสูตรวิทยาศาสตรบัณฑิต สาขาวิชาวิทยาศาสตร์ข้อมูลสุขภาพ \nปี 1 เทอม 1 ต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?", "วิศวะคอม/หลักสูตรปกติ ปี 3 เทอม 1\nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?", "วิศวกรรมคอมพิวเตอร์หลักสูตรนานาชาติ ปีที่1-2 เรียนอะไรบ้าง?", "วิชา CPE223/Digital มีวิชาตัวต่อมั้ยครับ?", "วิศวะคอม/หลักสูตรนานาชาติปี 4 เทอม 1\nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?", "วิชา CPE101/Exploration มีวิชาตัวต่อมั้ยครับ?", "วิศวกรรมคอมพิวเตอร์หลักสูตรเรสิเดนทอล คอเลจ ปีที่ 3-4 เรียนอะไรบ้าง?", "วิศวะคอม/หลักสูตรนานาชาติ ปี 3 เทอม 2\nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?", "วิชาภาษาอังกฤษ LNG มีเกณท์การเรื่มเรียนอย่างไร หลักสูตรปกติ ?", "วิชาภาคบังคับที่มีตัวต่อเนื่องมีอะไรบ้าง หลักสูตรนานาชาติ?", "วิศวะคอม/หลักสูตรปกติ  ปี 3 เทอม 2\nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?", "วิศวกรรมคอมพิวเตอร์หลักสูตรวิทยาศาตร์ข้อมูลสุขภาพ ปีที่1-2 เรียนอะไรบ้าง?", "วิชา CPE231/Database มีวิชาตัวต่อมั้ยครับ?", "วิชาภาคบังคับที่มีวิชาต่อเนื่อง  หลักสูตรวิทยาศาสตรบัณฑิต สาขาวิชาวิทยาศาสตร์ข้อมูลสุขภาพ \nมีอะไรบ้าง ?", "วิชาภาษาอังกฤษ LNG มีเกณท์การเรื่มเรียนอย่างไร หลักสูตรนานาชาติ ?", "วิศวกรรมคอมพิวเตอร์หลักสูตรปกติ ปีที่ 3 เรียนอะไรบ้าง?", "วิศวะคอม/หลักสูตรปกติ  ปี 2 เทอม 2\nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?", "วิศวะคอม/หลักสูตรปกติ ปี 1 เทอม 2\nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?", "วิศวะคอมหลักสูตรปกติมีระยะเวลาในการศึกษากี่ปี?", "วิศวะคอม/หลักสูตรวิทยาศาสตรบัณฑิต สาขาวิชาวิทยาศาสตร์ข้อมูลสุขภาพ \nปี 2 เทอม 1 ต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?", "วิชา CPE213/Data model วิชาตัวต่อมั้ยครับ?", "วิศวะคอม/หลักสูตรวิทยาศาสตรบัณฑิต สาขาวิชาวิทยาศาสตร์ข้อมูลสุขภาพ \nปี 1 เทอม 2 ต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?", "วิศวะคอม/ มีทั้งหมดกี่หลักสูตร? มีหลักสูตรอะไรบ้าง?", "วิศวะคอมหลักสูตรนานาชาติมีอัตราค่าเรียนเท่าไหร่", "วิศวะคอม/หลักสูตรวิทยาศาสตรบัณฑิต สาขาวิชาวิทยาศาสตร์ข้อมูลสุขภาพ \nปี 4 เทอม 1 ต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?", "วิศวกรรมคอมพิวเตอร์หลักสูตรนานาชาติ ปีที่ 4 เรียนอะไรบ้าง?", "วิชา CPE122/Circuits มีวิชาตัวต่อมั้ยครับ?", "วิศวะคอม/หลักสูตรปกติ  ปี 4 เทอม 2\nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?", "วิศวกรรมคอมพิวเตอร์หลักสูตรนานาชาติ ปีที่ 3 เรียนอะไรบ้าง?", "วิศวะคอม/หลักสูตรนานาชาติ ปี 3 เทอม 1\nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?", "วิชา CPE329/Business intelligence มีวิชาตัวต่อมั้ยครับ?", "วิศวะคอม/หลักสูตรวิทยาศาสตรบัณฑิต สาขาวิชาวิทยาศาสตร์ข้อมูลสุขภาพ \nปี 2 เทอม 2 ต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?", "วิศวกรรมคอมพิวเตอร์หลักสูตรปกติ ปีที่ 4 เรียนอะไรบ้าง?", "วิศวะคอม/หลักสูตรปกติ ปี 1 เทอม 1 \nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?"],
"inputQuestion":"วิศวคอมมีหลักสูตรอะไรบ้าง"
    }